In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,from_utc_timestamp
from pyspark.sql.window import Window
from pyspark.sql.functions import lag,sum,first

#file_loc='dbfs:/FileStore/tables/click_stream.txt'

spark = SparkSession.builder.appName('click-stream').getOrCreate()
sc = spark.sparkContext

data_list = [("U1","2023-01-01T11:00:00Z"),("U1","2023-01-01T11:15:00Z"),("U1","2023-01-01T12:00:00Z"),("U1","2023-01-01T12:20:00Z"),("U1","2023-01-01T15:00:00Z"),("U2","2023-01-01T11:00:00Z"),("U2","2023-01-02T11:00:00Z"),("U2","2023-01-02T11:25:00Z"),("U2","2023-01-02T11:50:00Z"),("U2","2023-01-02T12:15:00Z"),("U2","2023-01-02T12:40:00Z"),("U2","2023-01-02T13:05:00Z"),("U2","2023-01-02T13:20:00Z")]
df = sc.parallelize(data_list).toDF(["userid","login_time"])
print(df.show())
print(df.printSchema)
df1=df.withColumn('click_time',from_utc_timestamp(col('login_time'),'UTC')).drop('login_time')
print(df1.show())
df2=df1.withColumn("lag_click_time",lag('click_time',1).over(Window.partitionBy('userid').orderBy('click_time'))).drop('login_time').withColumn("time_diff",(col("click_time").cast('long')-col('lag_click_time').cast('long'))/(60)).fillna(0).withColumn("is_new_session", when(col("time_diff") >30 , 1).otherwise(0) ).withColumn("temp_session_id", sum(col("is_new_session")).over(Window.partitionBy("userid").orderBy("click_time"))).withColumn("first_click_time", first(col("click_time")).over(Window.partitionBy("userid", "temp_session_id").orderBy("click_time"))).withColumn("time_diff2", ((col("click_time").cast("long") - col("first_click_time").cast("long"))/(60 * 60 * 2)).cast("int")).withColumn("session_id", (col("time_diff2") + col("temp_session_id")))
df2.show()
df2.drop("lag_click_time","time_diff", "is_new_session", "temp_session_id", "first_click_time", "time_diff2" ).show()

+------+--------------------+
|userid|          login_time|
+------+--------------------+
|    U1|2023-01-01T11:00:00Z|
|    U1|2023-01-01T11:15:00Z|
|    U1|2023-01-01T12:00:00Z|
|    U1|2023-01-01T12:20:00Z|
|    U1|2023-01-01T15:00:00Z|
|    U2|2023-01-01T11:00:00Z|
|    U2|2023-01-02T11:00:00Z|
|    U2|2023-01-02T11:25:00Z|
|    U2|2023-01-02T11:50:00Z|
|    U2|2023-01-02T12:15:00Z|
|    U2|2023-01-02T12:40:00Z|
|    U2|2023-01-02T13:05:00Z|
|    U2|2023-01-02T13:20:00Z|
+------+--------------------+

None
<bound method DataFrame.printSchema of DataFrame[userid: string, login_time: string]>
+------+-------------------+
|userid|         click_time|
+------+-------------------+
|    U1|2023-01-01 11:00:00|
|    U1|2023-01-01 11:15:00|
|    U1|2023-01-01 12:00:00|
|    U1|2023-01-01 12:20:00|
|    U1|2023-01-01 15:00:00|
|    U2|2023-01-01 11:00:00|
|    U2|2023-01-02 11:00:00|
|    U2|2023-01-02 11:25:00|
|    U2|2023-01-02 11:50:00|
|    U2|2023-01-02 12:15:00|
|    U2|2023-01-02 12